In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from collections import Counter


In [3]:
data = pd.read_csv('data.csv')
data.head(5)

,imdb_id,popularity,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year
0,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015
1,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015
2,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015
3,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292,7.5,2015
4,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,2947,7.3,2015


# Предобработка датасета

In [4]:
answer_ls = [] # создадим список с ответами. сюда будем добавлять ответы по мере прохождения теста
# сюда можем вписать создание новых колонок в датасете

#### Добавим новые признаки в датасет и обработваем текущие

In [5]:
from enum import Enum

class Months(Enum):
    JAN = 1
    FEB = 2
    MAR = 3
    APR = 4
    MAY = 5
    JUN = 6
    JUL = 7
    AUG = 8
    SEP = 9
    OCT = 10
    NOV = 11
    DEC = 12
    
class Seasons(Enum):
    WINTER = [12,1,2]
    SPRING = [3,4,5]
    SUMMER = [6,7,8]
    AUTUMN = [9,10,11]
    
    def get_season(s):
        for season in Seasons:
            if s in season.value:
                return season.name

# Чистая прибыль
data['revenue_total'] = data.revenue-data.budget
data.release_date = pd.to_datetime(data.release_date)
# Месяц и сесон выхода фильма
data['release_month'] = data.release_date.apply(lambda d: Months(d.month).name)
data['release_season'] = data.release_date.apply(lambda d: Seasons.get_season(d.month))
# Количество символов и слов в названии фильма
data['title_len'] = data.original_title.apply(lambda t: len(t))
data['words_len'] = data.original_title.apply(lambda s: len(s.split()))

data.index.name = 'movie_id'

# Класс для получения датафрэйма, где признаки разбиты через разделитель (|)
class PrepareDataset():
    
    def __init__(self, dataframe):
        self.df = dataframe
        
    def get_feature_list(self, feature_name, value_counts=True, **kwargs):
        if value_counts is True:
            feature_df = self.df[feature_name].str.split('|', expand=True).stack().value_counts()
            columns = ['name','feature_count']
            
        else:
            feature_df = self.df[feature_name].str.split('|', expand=True).stack()
            columns = ['movie_id','feature_count','name']
        feature_df = feature_df.reset_index()
        feature_df.columns = columns
        
        if len(kwargs) > 0:
            if kwargs.get('add_feature'):
                add_features = kwargs.get('add_feature')
                feature_df = feature_df.merge(self.df[add_features], on='movie_id', how='left')

        return feature_df
    
    def get_max_feature(self, feature_df, feature='feature_count'):
        return feature_df[feature_df[feature] == feature_df[feature].max()]
    
    def get_min_feature(self, feature_df, feature='feature_count'):
        return feature_df[feature_df[feature] == feature_df[feature].min()]
    
    def get_max_value(self, feature):
        return self.df[self.df[feature] == self.df[feature].max()]
    
    def get_min_value(self, feature):
        return self.df[self.df[feature] == self.df[feature].min()]



# 1. У какого фильма из списка самый большой бюджет?
Варианты ответов:
1. The Dark Knight Rises (tt1345836)
2. Spider-Man 3 (tt0413300)
3. Avengers: Age of Ultron (tt2395427)
4. The Warrior's Way	(tt1032751)
5. Pirates of the Caribbean: On Stranger Tides (tt1298650)

In [6]:
top_budget = data[data.budget == data.budget.max()].original_title
print(top_budget)

# тут вводим ваш ответ и добавлем в его список ответов (сейчас для примера стоит "1")
answer_ls.append(4)

movie_id
491    The Warrior's Way
Name: original_title, dtype: object


# 2. Какой из фильмов самый длительный (в минутах)
1. The Lord of the Rings: The Return of the King	(tt0167260)
2. Gods and Generals	(tt0279111)
3. King Kong	(tt0360717)
4. Pearl Harbor	(tt0213149)
5. Alexander	(tt0346491)

In [7]:
runtime = data[data.runtime == data.runtime.max()].original_title
print(runtime)

answer_ls.append(2)

movie_id
1158    Gods and Generals
Name: original_title, dtype: object


# 3. Какой из фильмов самый короткий (в минутах)
Варианты ответов:

1. Home on the Range	tt0299172
2. The Jungle Book 2	tt0283426
3. Winnie the Pooh	tt1449283
4. Corpse Bride	tt0121164
5. Hoodwinked!	tt0443536

In [8]:
short_movie = data[data.runtime == data.runtime.min()].original_title
print(short_movie)

answer_ls.append(3)

movie_id
769    Winnie the Pooh
Name: original_title, dtype: object


# 4. Средняя длительность фильма?

Варианты ответов:
1. 115
2. 110
3. 105
4. 120
5. 100


In [9]:
avg_runtime = round(data.runtime.mean())
print(avg_runtime)

answer_ls.append(2)

110


# 5. Средняя длительность фильма по медиане?
Варианты ответов:
1. 106
2. 112
3. 101
4. 120
5. 115




In [10]:
avg_runtime = round(data.runtime.median())
print(avg_runtime)

answer_ls.append(1)

106


# 6. Какой самый прибыльный фильм?
Варианты ответов:
1. The Avengers	tt0848228
2. Minions	tt2293640
3. Star Wars: The Force Awakens	tt2488496
4. Furious 7	tt2820852
5. Avatar	tt0499549

In [11]:
most_revenue = data[data.revenue_total == data.revenue_total.max()].original_title
print(most_revenue)

answer_ls.append(5)

movie_id
239    Avatar
Name: original_title, dtype: object


# 7. Какой фильм самый убыточный?
Варианты ответов:
1. Supernova tt0134983
2. The Warrior's Way tt1032751
3. Flushed Away	tt0424095
4. The Adventures of Pluto Nash	tt0180052
5. The Lone Ranger	tt1210819

In [12]:
most_negative_revenue = data[data.revenue_total == data.revenue_total.min()].original_title
print(most_negative_revenue)

answer_ls.append(2)

movie_id
491    The Warrior's Way
Name: original_title, dtype: object


# 8. Сколько всего фильмов в прибыли?
Варианты ответов:
1. 1478
2. 1520
3. 1241
4. 1135
5. 1398


In [13]:
positive_revenue_count = data[data.revenue > data.budget]['original_title'].count()
print(positive_revenue_count)

answer_ls.append(1)

1478


# 9. Самый прибыльный фильм в 2008 году?
Варианты ответов:
1. Madagascar: Escape 2 Africa	tt0479952
2. Iron Man	tt0371746
3. Kung Fu Panda	tt0441773
4. The Dark Knight	tt0468569
5. Mamma Mia!	tt0795421

In [14]:
most_revenue_2008 = data[data.release_year == 2008]
most_revenue_2008 = most_revenue_2008[most_revenue_2008.revenue_total == most_revenue_2008.revenue_total.max()].original_title
print(most_revenue_2008)

answer_ls.append(4)

movie_id
600    The Dark Knight
Name: original_title, dtype: object


# 10. Самый убыточный фильм за период с 2012 по 2014 (включительно)?
Варианты ответов:
1. Winter's Tale	tt1837709
2. Stolen	tt1656186
3. Broken City	tt1235522
4. Upside Down	tt1374992
5. The Lone Ranger	tt1210819


In [15]:
most_negative_2012_to_2014 = data[data.release_year.between(2012,2014)]
most_negative_2012_to_2014 = most_negative_2012_to_2014[most_negative_2012_to_2014.revenue_total == most_negative_2012_to_2014.revenue_total.min()].original_title

print(most_negative_2012_to_2014)
answer_ls.append(5)

movie_id
1246    The Lone Ranger
Name: original_title, dtype: object


# 11. Какого жанра фильмов больше всего?
Варианты ответов:
1. Action
2. Adventure
3. Drama
4. Comedy
5. Thriller

In [16]:
dataset = PrepareDataset(data)
genres = dataset.get_feature_list('genres')
top_genres = dataset.get_max_feature(genres)

print(top_genres)
answer_ls.append(3)

    name  feature_count
0  Drama            782


# 12. Какого жанра среди прибыльных фильмов больше всего?
Варианты ответов:
1. Drama
2. Comedy
3. Action
4. Thriller
5. Adventure

In [17]:
most_revenue_genres = PrepareDataset(data[data.revenue > data.budget])
revenue_genres = most_revenue_genres.get_feature_list('genres')
top_revenue_genres = most_revenue_genres.get_max_feature(revenue_genres)

print(top_revenue_genres)
answer_ls.append(1)

    name  feature_count
0  Drama            560


# 13. Кто из режиссеров снял больше всего фильмов?
Варианты ответов:
1. Steven Spielberg
2. Ridley Scott 
3. Steven Soderbergh
4. Christopher Nolan
5. Clint Eastwood

In [18]:
directors = PrepareDataset(data)
directors_df = directors.get_feature_list('director')
directors_df_most_movies = directors.get_max_feature(directors_df)

print(directors_df_most_movies)
answer_ls.append(3)

                name  feature_count
0  Steven Soderbergh             13


# 14. Кто из режиссеров снял больше всего Прибыльных фильмов?
Варианты ответов:
1. Steven Soderbergh
2. Clint Eastwood
3. Steven Spielberg
4. Ridley Scott
5. Christopher Nolan

In [19]:
directors_revenue = PrepareDataset(data[data.revenue > data.budget])
directors_revenue_df = directors_revenue.get_feature_list('director')
directors_revenue_top = directors_revenue.get_max_feature(directors_revenue_df)

print(directors_revenue_top)
answer_ls.append(4)

           name  feature_count
0  Ridley Scott             12


# 15. Кто из режиссеров принес больше всего прибыли?
Варианты ответов:
1. Steven Spielberg
2. Christopher Nolan
3. David Yates
4. James Cameron
5. Peter Jackson


In [20]:
directors = PrepareDataset(data)
directors_df = directors.get_feature_list('director', False, add_feature=['revenue_total'])
directors_df_most_revenue = directors_df.groupby('name', as_index=False)['revenue_total'].sum()
directors_df_most_revenue = directors.get_max_feature(directors_df_most_revenue, 'revenue_total')

print(directors_df_most_revenue)
answer_ls.append(5)

              name  revenue_total
727  Peter Jackson     5202593685


# 16. Какой актер принес больше всего прибыли?
Варианты ответов:
1. Emma Watson
2. Johnny Depp
3. Michelle Rodriguez
4. Orlando Bloom
5. Rupert Grint

In [21]:
actors = PrepareDataset(data)
actors_df = actors.get_feature_list('cast', False, add_feature=['revenue_total'])
actors_df = actors_df.groupby('name', as_index=False)['revenue_total'].sum()
actors_df = actors.get_max_feature(actors_df, 'revenue_total')

print(actors_df)
answer_ls.append(1)

             name  revenue_total
1034  Emma Watson     6666245597


# 17. Какой актер принес меньше всего прибыли в 2012 году?
Варианты ответов:
1. Nicolas Cage
2. Danny Huston
3. Kirsten Dunst
4. Jim Sturgess
5. Sami Gayle

In [22]:
actors = PrepareDataset(data[data.release_year == 2012])
actors_df = actors.get_feature_list('cast', False, add_feature=['revenue_total'])
actors_df = actors_df.groupby('name', as_index=False)['revenue_total'].sum()
actors_df = actors.get_min_feature(actors_df, 'revenue_total')

print(actors_df)
answer_ls.append(3)

              name  revenue_total
253  Kirsten Dunst      -68109207


# 18. Какой актер снялся в большем количестве высокобюджетных фильмов? (в фильмах где бюджет выше среднего по данной выборке)
Варианты ответов:
1. Tom Cruise
2. Mark Wahlberg 
3. Matt Damon
4. Angelina Jolie
5. Adam Sandler

In [23]:
actors = PrepareDataset(data[data.budget > data.budget.mean()])
actors_df = actors.get_feature_list('cast')
actors_df = actors.get_max_feature(actors_df)

print(actors_df)
answer_ls.append(3)

         name  feature_count
0  Matt Damon             18


# 19. В фильмах какого жанра больше всего снимался Nicolas Cage?  
Варианты ответа:
1. Drama
2. Action
3. Thriller
4. Adventure
5. Crime

In [24]:
nicolas_cage = PrepareDataset(data[data.cast.str.contains('Nicolas Cage')])
nicolas_cage_genres = nicolas_cage.get_feature_list('genres')
nicolas_cage_genres = nicolas_cage.get_max_feature(nicolas_cage_genres)

print(nicolas_cage_genres)
answer_ls.append(2)

     name  feature_count
0  Action             17


# 20. Какая студия сняла больше всего фильмов?
Варианты ответа:
1. Universal Pictures (Universal)
2. Paramount Pictures
3. Columbia Pictures
4. Warner Bros
5. Twentieth Century Fox Film Corporation

In [25]:
production_companies = PrepareDataset(data)
production_companies_df = production_companies.get_feature_list('production_companies')
production_companies_df = production_companies.get_max_feature(production_companies_df)

print(production_companies_df)
answer_ls.append(1)

                 name  feature_count
0  Universal Pictures            173


# 21. Какая студия сняла больше всего фильмов в 2015 году?
Варианты ответа:
1. Universal Pictures
2. Paramount Pictures
3. Columbia Pictures
4. Warner Bros
5. Twentieth Century Fox Film Corporation

In [26]:
production_companies = PrepareDataset(data[data.release_year == 2015])
production_companies_df = production_companies.get_feature_list('production_companies')
production_companies_df = production_companies.get_max_feature(production_companies_df)

print(production_companies_df)
answer_ls.append(4)

           name  feature_count
0  Warner Bros.             12


# 22. Какая студия заработала больше всего денег в жанре комедий за все время?
Варианты ответа:
1. Warner Bros
2. Universal Pictures (Universal)
3. Columbia Pictures
4. Paramount Pictures
5. Walt Disney

In [27]:
production_companies = PrepareDataset(data[data.genres.str.contains('Comedy')])
production_companies_df = production_companies.get_feature_list('production_companies', False, add_feature=['revenue_total'])
production_companies_df = production_companies_df.groupby('name', as_index=False)['revenue_total'].sum()
production_companies_df = production_companies.get_max_feature(production_companies_df, 'revenue_total')

print(production_companies_df)
answer_ls.append(2)

                   name  revenue_total
644  Universal Pictures     8961545581


# 23. Какая студия заработала больше всего денег в 2012 году?
Варианты ответа:
1. Universal Pictures (Universal)
2. Warner Bros
3. Columbia Pictures
4. Paramount Pictures
5. Lucasfilm

In [28]:
production_companies = PrepareDataset(data[data.release_year == 2012])
production_companies_df = production_companies.get_feature_list('production_companies', False, add_feature=['revenue_total'])
production_companies_df = production_companies_df.groupby('name', as_index=False)['revenue_total'].sum()
production_companies_df = production_companies.get_max_feature(production_companies_df, 'revenue_total')

print(production_companies_df)
answer_ls.append(3)

                 name  revenue_total
47  Columbia Pictures     2501406608


# 24. Самый убыточный фильм от Paramount Pictures
Варианты ответа:

1. K-19: The Widowmaker tt0267626
2. Next tt0435705
3. Twisted tt0315297
4. The Love Guru tt0811138
5. The Fighter tt0964517

In [29]:
production_companies = PrepareDataset(data[data.production_companies.str.contains('Paramount Pictures')])
production_companies = production_companies.get_min_value('revenue_total').original_title

print(production_companies)
answer_ls.append(1)

movie_id
926    K-19: The Widowmaker
Name: original_title, dtype: object


# 25. Какой Самый прибыльный год (заработали больше всего)?
Варианты ответа:
1. 2014
2. 2008
3. 2012
4. 2002
5. 2015

In [30]:
most_revenue_year = PrepareDataset(data.groupby('release_year', as_index=False)['revenue_total'].sum())
most_revenue_year = most_revenue_year.get_max_value('revenue_total')

print(most_revenue_year)
answer_ls.append(5)

    release_year  revenue_total
15          2015    18668572378


# 26. Какой Самый прибыльный год для студии Warner Bros?
Варианты ответа:
1. 2014
2. 2008
3. 2012
4. 2010
5. 2015

In [31]:
most_revenue_warnerbros = PrepareDataset(data[data.production_companies.str.contains('Warner Bros')].groupby('release_year', as_index=False)['revenue_total'].sum())

print(most_revenue_warnerbros.get_max_value('revenue_total'))
answer_ls.append(1)

    release_year  revenue_total
14          2014     2295464519


# 27. В каком месяце за все годы суммарно вышло больше всего фильмов?
Варианты ответа:
1. Январь
2. Июнь
3. Декабрь
4. Сентябрь
5. Май

In [32]:
top_month = PrepareDataset(data.groupby('release_month', as_index=False)['original_title'].count())

print(top_month.get_max_value('original_title'))
answer_ls.append(4)

   release_month  original_title
11           SEP             227


# 28. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)
Варианты ответа:
1. 345
2. 450
3. 478
4. 523
5. 381

In [33]:
total_season = data[data.release_season == 'SUMMER']['original_title'].count()

print(total_season)
answer_ls.append(2)

450


# 29. Какой режисер выпускает (суммарно по годам) больше всего фильмов зимой?
Варианты ответов:
1. Steven Soderbergh
2. Christopher Nolan
3. Clint Eastwood
4. Ridley Scott
5. Peter Jackson

In [34]:
directors = PrepareDataset(data[data.release_season == 'WINTER'])
directors_df = directors.get_feature_list('director')

print(directors.get_max_feature(directors_df))
answer_ls.append(5)

            name  feature_count
0  Peter Jackson              7


# 30. Какой месяц чаще всего по годам самый прибыльный?
Варианты ответа:
1. Январь
2. Июнь
3. Декабрь
4. Сентябрь
5. Май

In [35]:
month_avg_revenue = data.groupby(['release_year','release_month'], as_index=False)['revenue_total'].sum()
month_avg_revenue = month_avg_revenue.loc[month_avg_revenue.groupby('release_year')['revenue_total'].idxmax()]
top_avg_m = month_avg_revenue.groupby('release_month', as_index=False)['revenue_total'].count()

print(top_avg_m[top_avg_m.revenue_total == top_avg_m.revenue_total.max()])
answer_ls.append(2)

  release_month  revenue_total
2           JUN              7


# 31. Названия фильмов какой студии в среднем самые длинные по количеству символов?
Варианты ответа:
1. Universal Pictures (Universal)
2. Warner Bros
3. Jim Henson Company, The
4. Paramount Pictures
5. Four By Two Productions

In [36]:
studios_title_len_dat = PrepareDataset(data)
studios_title_len = studios_title_len_dat.get_feature_list('production_companies', False, add_feature=['title_len'])
studios_title_len = studios_title_len.groupby('name', as_index=False)['title_len'].mean()
studio_title_len_max = studios_title_len_dat.get_max_feature(studios_title_len, 'title_len')

print(studio_title_len_max)
answer_ls.append(5)

                        name  title_len
584  Four By Two Productions       83.0


# 32. Названия фильмов какой студии в среднем самые длинные по количеству слов?
Варианты ответа:
1. Universal Pictures (Universal)
2. Warner Bros
3. Jim Henson Company, The
4. Paramount Pictures
5. Four By Two Productions

In [37]:
studios_title_len_dat = PrepareDataset(data)
studios_title_len = studios_title_len_dat.get_feature_list('production_companies', False, add_feature=['words_len'])
studios_title_len = studios_title_len.groupby('name', as_index=False)['words_len'].mean()
studio_title_len_max = studios_title_len_dat.get_max_feature(studios_title_len, 'words_len')

print(studio_title_len_max)
answer_ls.append(5)

                        name  words_len
584  Four By Two Productions       12.0


# 33. Сколько разных слов используется в названиях фильмов?(без учета регистра)
Варианты ответа:
1. 6540
2. 1002
3. 2461
4. 28304
5. 3432

In [38]:
from collections import Counter
result = Counter(" ".join(data['original_title'].apply(lambda s: s.lower()).values.tolist()).split()).items()
print(len(result))

answer_ls.append(3)

2461


# 34. Какие фильмы входят в 1 процент лучших по рейтингу?
Варианты ответа:
1. Inside Out, Gone Girl, 12 Years a Slave
2. BloodRayne, The Adventures of Rocky & Bullwinkle
3. The Lord of the Rings: The Return of the King, Upside Down
4. 300, Lucky Number Slevin
5. Upside Down, 300, Inside Out, The Lord of the Rings: The Return of the King

In [39]:
top_movies = data[data.vote_average > data.vote_average.mean()].groupby('original_title', as_index=False)['vote_average'].last().sort_values(by=('vote_average'), ascending=False)
percent = round(len(top_movies)/100)
print(top_movies.head(percent))

answer_ls.append(1)

                                    original_title  vote_average
691                                The Dark Knight           8.1
335                                   Interstellar           8.0
553                                           Room           8.0
331                                     Inside Out           8.0
747                             The Imitation Game           8.0
1                                 12 Years a Slave           7.9
718                       The Grand Budapest Hotel           7.9
264                                      Gone Girl           7.9
775  The Lord of the Rings: The Return of the King           7.9


# 35. Какие актеры чаще всего снимаются в одном фильме вместе
Варианты ответа:
1. Johnny Depp & Helena Bonham Carter
2. Hugh Jackman & Ian McKellen
3. Vin Diesel & Paul Walker
4. Adam Sandler & Kevin James
5. Daniel Radcliffe & Rupert Grint

In [40]:
def get_actor(actor):
    played_with = data[data.cast.str.contains(actor)]['cast'].values.tolist()
    actors = ",".join(played_with).replace('|',',').split(',')
    actors_list = list(filter(lambda s: s != actor, actors))
    
    return Counter(actors_list).most_common(2)
    
actors = PrepareDataset(data)
actors_df = actors.get_feature_list('cast')
actors_df['most_played'] = actors_df.name.apply(get_actor)
actors_df['most_played_count'] = actors_df.most_played.apply(lambda s: s[0][1])

answer_ls.append(1)
print(actors_df[actors_df.most_played_count == actors_df.most_played_count.max()].most_played)

107    [(Daniel Radcliffe, 8), (Rupert Grint, 8)]
136         [(Rupert Grint, 8), (Emma Watson, 8)]
205     [(Daniel Radcliffe, 8), (Emma Watson, 8)]
Name: most_played, dtype: object


# 36. У какого из режиссеров выше вероятность выпустить фильм в прибыли? (5 баллов)101
Варианты ответа:
1. Quentin Tarantino
2. Steven Soderbergh
3. Robert Rodriguez
4. Christopher Nolan
5. Clint Eastwood

In [41]:
directors_data = PrepareDataset(data)
directors = directors_data.get_feature_list('director')
directors['revenue_count'] = directors.name.apply(lambda s: data[(data.director.str.contains(s, regex=False)) & (data.revenue > data.budget)]['budget'].count())
directors['percent_revenue'] = directors.revenue_count/directors.feature_count*100
top_director = directors[(directors.name == 'Quentin Tarantino') | (directors.name == 'Steven Soderbergh') | (directors.name == 'Robert Rodriguez') | (directors.name == 'Christopher Nolan') | (directors.name == 'Clint Eastwood')]

print(top_director[top_director.percent_revenue == top_director.percent_revenue.max()])
answer_ls.append(4)

                 name  feature_count  revenue_count  percent_revenue
17  Christopher Nolan              8              8            100.0


# Submission

In [42]:
len(answer_ls)

36

In [43]:
pd.DataFrame({'Id':range(1,len(answer_ls)+1), 'Answer':answer_ls}, columns=['Id', 'Answer'])

,Id,Answer
0,1,4
1,2,2
2,3,3
3,4,2
4,5,1
5,6,5
6,7,2
7,8,1
8,9,4
9,10,5
